<a href="https://colab.research.google.com/github/astorguy/learn_ngspice/blob/main/notebooks/divider/divider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Example 1: Introduction and First Ngspice Simulation***

# Introduction
This is a hands-on introduction to [Ngspice](https://ngspice.sourceforge.io/), using [Google Colaboratory](https://colab.research.google.com/). A "Colab" is essentially a [Jupyter](https://jupyter.org/) notebook in the Google ecosystem. Jupyter notesbooks interactively generate, process, and visualize data using Python. We will be using the [PyOPUS](https://fides.fe.uni-lj.si/pyopus/) package to facilitate interaction with Ngspice.

Our objective is to learn how to use Ngspice for circuit simulation and analysis. We'll go through progressively more complicated circuits, analyses, and data processing, explaining Ngspice concepts along the way.

# Installations, Imports, Initializations

In [1]:
# Here are all the imports except PyOPUS, which must to be built first
import os
import shutil, subprocess, sys
from pathlib import Path
from types import SimpleNamespace

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# create paths namespace; we use it to keep all the paths organized
paths = SimpleNamespace()

# set the workspace: it's different for Colab, Github Codespace, Jupyter
paths.workspace = Path.cwd()

#### Install Ngspice

In [3]:
subprocess.run(["sudo", "apt-get", "update"], check=True)
subprocess.run(["sudo", "apt-get", "install", "-y", "ngspice"], check=True)
paths.ngspice = Path("/usr/bin/ngspice")

Hit:1 http://deb.debian.org/debian trixie InRelease
Get:2 http://deb.debian.org/debian trixie-updates InRelease [47.3 kB]
Hit:3 http://deb.debian.org/debian-security trixie-security InRelease
Fetched 47.3 kB in 0s (488 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
ngspice is already the newest version (44.2+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.


#### Download and install PyOPUS

In [4]:
URL = "https://fides.fe.uni-lj.si/pyopus/download/0.12/"
PYOPUS_SRC = "pyopus-0.12"
TARFILE = f"{PYOPUS_SRC}.tar.gz"

# Download, extract, and build PyOPUS from archive
subprocess.run(["wget", "-q", URL + TARFILE], check=True)
subprocess.run(["tar", "-xzf", TARFILE], check=True)
subprocess.check_call([sys.executable, "-m", "pip", "install", f"./{PYOPUS_SRC}"])

Path(TARFILE).unlink() # Delete downloaded tar file
shutil.rmtree(PYOPUS_SRC)   # Delete extracted source directory

Defaulting to user installation because normal site-packages is not writeable
Processing ./pyopus-0.12
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for PyOPUS: filename=pyopus-0.12-cp314-cp314-linux_x86_64.whl size=4828244 sha256=5eef09cac3e7009559aced8c807efd6407efbc4c0f65c916315ee8efaee86851
  Stored in directory: /home/vscode/.cache/pip/wheels/53/d8/96/d711ecb9a9ddd253f74950d71d3840d37765eba87152b18ebe
Successfully built PyOPUS
  Attempting uninstall: PyOPUS
    Found existing installation: PyOPUS 0.12
    Uninstalling PyOPUS-0.12:
      Successfully uninstalled PyOPUS-0.12


#### pyopus imports and definitions

In [5]:
from pyopus.simulator.ngspice import Ngspice
from pyopus.simulator.ngspice import an_tran
from pyopus.evaluator import measure

sim = Ngspice(binary = paths.ngspice, debug=0)  # define simulator

## First Ngspice Simulation
Eventually, we will execute Ngspice from Python. But for this first example we will start by running it directly from the command line. This is the traditional method for running Ngspice. However, it's tedious and error-prone, so we'll introduce Python and PyOPUS in subsequent examples.

## The `dut.cir` Netlist
Ngspice requires a file called a *netlist*. This file contains should have at least the following items:
- The first line ***must*** be the title line. It is treated as a comment
- The last line ***must*** be: `.end`
- In the most general cases, individual components of the circuit are each listed on their own lines.
- The type of simulation is specified with a line starting with a `.` (period)

There are many other aspects to netlists, which we'll cover as needed.

<img src="https://raw.githubusercontent.com/astorguy/learn_ngspice/main/notebooks/divider/assets/dut.svg" width="360" height="auto">

### Circuit components
Every component in the circuit starts with a letter (note that Ngspice is generally case insensitive). Our circuit has three components.
- Any line that begins with an `R` is a resistor. The first word must have one or more additional characters to make it unique. In the first case we use a `1` to make it `R1`. The next words are the connection nodes. For a resistor there are two nodes. The last word is the resistor value. In the case of `R1`, the value is `1k` ohms. For `R1` the connections are `in` and `out`. For `R2` the connections are `out` and `0`, and the value is `2k` ohms.
- Any line that begins with a `V` is a voltage source. Again the first word needs one or more characters to make it unique. The second word is the positive connection node. The third word is the negative connection node. The remaining words describe the type of voltage source, with many different options. In our case, we have a DC voltage source of `10` volts.

### Simulations specification
There are several different types of simulations available in Ngspice. For our simple example, we are performing an "operating point" (`.op`).

The following code creates our netlist and writes it into a file named `dut.cir` (device under test).

***Note:*** *Netlists are normally generated directly from a schematic capture, such as [KiCad](https://www.kicad.org/).*

In [6]:
this_netlist = """
* First line is always the title
V1 in 0 DC 10
R1 in out 1k
R2 out 0 2k
.op
.end
"""
paths.dut = Path("dut.cir")
paths.dut.write_text(this_netlist)

82

The following Ngspice command has the `-b` switch for "batch mode" and the name of the file to read in.

In [7]:
subprocess.run(["ngspice", "-b", "dut.cir"], check=True)


Note: No compatibility mode selected!


Circuit: 

Doing analysis at TEMP = 27.000000 and TNOM = 27.000000

Using SPARSE 1.3 as Direct Linear Solver

No. of Data Rows : 1
	Node                                  Voltage
	----                                  -------
	----	-------
	out                              6.666667e+00
	in                               1.000000e+01

	Source	Current
	------	-------

	v1#branch                        -3.33333e-03

 Resistor models (Simple linear resistor)
      model                     R

        rsh                     0
     narrow                     0
      short                     0
        tc1                     0
        tc2                     0
        tce                     0
       defw                 1e-05
          l                 1e-05
         kf                     0
         af                     0
          r                     0
     bv_max                 1e+99
         lf                     1
         wf              

CompletedProcess(args=['ngspice', '-b', 'dut.cir'], returncode=0)

Ngspice generates an overload of information. However, if we look closely at the output log, we can see on about line 13 the voltage at `out` is `6.666667e+00` volts.

***We have completed our first Ngspice simulation!***

### ***Example 2: Integrating Ngspice into Python with PyOPUS***

## Introduction
We are going to simulate the same divider circuit, but this time control Ngspice within Python using the PyOPUS package. At first, this will seem like a more complicated way to use Ngspice. However, it gives us complete control within Python, enabling us to vary parameters, optimize, and generate innovative data presentations.

For this simple, we need only three steps:

1. Create a job list
2. Simulate all the jobs in the job list
3. Display the results

We create the DUT netlist again. But this time we leave off the title line, analysis line, and .end line. PyOPUS adds those in for us.

In [8]:
this_netlist = """
V1 in 0 DC 10
R1 in out 1k
R2 out 0 2k
"""

paths.dut = Path("dut.cir")
paths.dut.write_text(this_netlist)

40

## Job list
In our simple case, we only have one job in our list. Each job has seven possible dictionary key-value pairs. Some can be left blank if not needed.

In [9]:
jobList=[
    {
        'name': 'op1',
        'definitions': [{ 'file': str(paths.dut) }],    # which file to simulate
        'params': {},
        'options': {},
        'variables': {},
        'saves':['["i(v1)", "v(out)"]'],    # what outputs in our results
        'command': ('op()') # type of analysis to run (e.g. operating point)
    }
]

sim.setJobList(jobList) # send the job list to the simulator

## Simulate
The inner for loop runs through the job list we provided to the simulator. The simulator puts the jobs into groups. For most cases, Ngspice simulations will only have one group, which holds all the jobs. 

In [10]:
# Assume only one job group (index 0)
(jobIndices, status) = sim.runJobGroup(0)
for j in jobIndices:
    results = sim.readResults(j, status) # simulate

## Results

In [11]:
I_v1 = results.i('v1')
V_out = results.v('out')
print(f"The current through the resistors is {I_v1}")
print(f"The output voltage of the divider is {V_out}")

The current through the resistors is [-0.00333333]
The output voltage of the divider is [6.66666667]


### Clean-up
PyOPUS generates intermediate files for each group, that we can now delete.

In [12]:
sim.cleanup() # Removes intermediate files created by sim

## Final notes
This was a very simple circuit, with only a table of values and no post-processing. Subsequent examples will be more complex.